# Rossmann Store Sales Forecasting
## Machine Learning Zoomcamp 2025 - Midterm Project

**Objective**: Predict daily sales for Rossmann drugstore chains to optimize inventory management and business planning.

**Dataset**: Rossmann Store Sales from Kaggle Competition
- Historical sales data from 1,115 stores
- Time period: 2013-2015
- Features: Store attributes, promotions, holidays, competition data

**Business Value**:
- Reduce inventory costs through accurate demand forecasting
- Optimize staffing levels based on predicted sales volume
- Improve customer satisfaction by preventing stockouts
- Enable data-driven business planning

## 1. Environment Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

# Time series
from prophet import Prophet
import datetime as dt
from datetime import datetime, timedelta

# Utilities
import warnings
import joblib
import os
from pathlib import Path

# Settings
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
np.random.seed(42)

print("✅ Libraries imported successfully")

In [ ]:
# Load the datasets
data_path = Path('../data')

# Check if data files exist
required_files = ['train.csv', 'test.csv', 'store.csv']
missing_files = [f for f in required_files if not (data_path / f).exists()]

if missing_files:
    print(f"❌ Missing data files: {missing_files}")
    print("Please run: python ../data/download_data.py")
else:
    # Load datasets
    print("📊 Loading datasets...")
    
    train_df = pd.read_csv(data_path / 'train.csv')
    test_df = pd.read_csv(data_path / 'test.csv')
    store_df = pd.read_csv(data_path / 'store.csv')
    
    print(f"✅ Train data: {train_df.shape}")
    print(f"✅ Test data: {test_df.shape}")
    print(f"✅ Store data: {store_df.shape}")
    
    # Display basic info
    print("\n📈 Data loaded successfully!")

## 2. Initial Data Exploration

In [ ]:
# Examine the structure of each dataset
print("🔍 TRAIN DATASET OVERVIEW")
print("=" * 50)
print(f"Shape: {train_df.shape}")
print(f"Memory usage: {train_df.memory_usage().sum() / 1024**2:.2f} MB")
print("\nColumns:")
print(train_df.dtypes)

print("\n" + "=" * 50)
print("First 5 rows:")
display(train_df.head())

In [ ]:
print("🏪 STORE DATASET OVERVIEW")
print("=" * 50)
print(f"Shape: {store_df.shape}")
print("\nColumns:")
print(store_df.dtypes)
print("\n" + "=" * 50)
display(store_df.head())

# Check unique values for categorical columns
print("\n📊 Categorical Variables Summary:")
categorical_cols = ['StoreType', 'Assortment', 'PromoInterval']
for col in categorical_cols:
    if col in store_df.columns:
        print(f"{col}: {store_df[col].nunique()} unique values")
        print(f"  Values: {store_df[col].unique()}\n")

## 3. Data Quality Assessment

In [ ]:
def assess_data_quality(df, name):
    """Comprehensive data quality assessment"""
    
    print(f"🔍 DATA QUALITY ASSESSMENT - {name.upper()}")
    print("=" * 60)
    
    # Missing values
    missing_vals = df.isnull().sum()
    missing_pct = (missing_vals / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Missing Count': missing_vals,
        'Missing %': missing_pct
    }).sort_values('Missing Count', ascending=False)
    
    print("📊 Missing Values Analysis:")
    display(missing_df[missing_df['Missing Count'] > 0])
    
    # Data types and memory
    print(f"\n💾 Memory Usage: {df.memory_usage().sum() / 1024**2:.2f} MB")
    print(f"📏 Dataset Shape: {df.shape}")
    
    # Duplicates
    duplicates = df.duplicated().sum()
    print(f"🔄 Duplicate Rows: {duplicates}")
    
    return missing_df

# Assess each dataset
train_quality = assess_data_quality(train_df, 'Train')
store_quality = assess_data_quality(store_df, 'Store')
test_quality = assess_data_quality(test_df, 'Test')

In [ ]:
# Merge train data with store information
print("🔗 Merging train and store datasets...")
train_full = train_df.merge(store_df, on='Store', how='left')
print(f"✅ Merged dataset shape: {train_full.shape}")

# Convert date column
train_full['Date'] = pd.to_datetime(train_full['Date'])
train_full = train_full.sort_values(['Store', 'Date']).reset_index(drop=True)

print("\n📅 Date Range:")
print(f"From: {train_full['Date'].min()}")
print(f"To: {train_full['Date'].max()}")
print(f"Total Days: {(train_full['Date'].max() - train_full['Date'].min()).days}")

## 4. Target Variable Analysis

In [ ]:
# Sales distribution analysis
print("💰 SALES ANALYSIS")
print("=" * 50)

# Basic statistics
sales_stats = train_full['Sales'].describe()
print("📊 Sales Statistics:")
print(sales_stats)

# Check for zero sales
zero_sales = (train_full['Sales'] == 0).sum()
zero_sales_pct = (zero_sales / len(train_full)) * 100
print(f"\n🔍 Zero Sales Days: {zero_sales:,} ({zero_sales_pct:.2f}%)")

# Sales when store is open vs closed
print("\n🏪 Sales by Store Status:")
store_status = train_full.groupby('Open')['Sales'].agg(['count', 'mean', 'std']).round(2)
display(store_status)

In [ ]:
# Visualize sales distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Sales Distribution Analysis', fontsize=16, y=1.02)

# Filter out zero sales for better visualization
sales_nonzero = train_full[train_full['Sales'] > 0]['Sales']

# Histogram
axes[0, 0].hist(sales_nonzero, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Sales Distribution (Excluding Zero Sales)')
axes[0, 0].set_xlabel('Sales')
axes[0, 0].set_ylabel('Frequency')

# Box plot
axes[0, 1].boxplot(sales_nonzero)
axes[0, 1].set_title('Sales Box Plot')
axes[0, 1].set_ylabel('Sales')

# Log-scale histogram
axes[1, 0].hist(np.log1p(sales_nonzero), bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Log(Sales + 1) Distribution')
axes[1, 0].set_xlabel('Log(Sales + 1)')
axes[1, 0].set_ylabel('Frequency')

# Sales by day of week
daily_sales = train_full[train_full['Sales'] > 0].groupby('DayOfWeek')['Sales'].mean()
axes[1, 1].bar(range(1, 8), daily_sales, color='coral', alpha=0.7)
axes[1, 1].set_title('Average Sales by Day of Week')
axes[1, 1].set_xlabel('Day of Week (1=Monday)')
axes[1, 1].set_ylabel('Average Sales')
axes[1, 1].set_xticks(range(1, 8))

plt.tight_layout()
plt.show()

print(f"\n📈 Sales range: {sales_nonzero.min():,.0f} - {sales_nonzero.max():,.0f}")
print(f"📊 Sales variance: {sales_nonzero.var():,.0f}")
print(f"📏 Sales std: {sales_nonzero.std():,.0f}")

## 5. Time Series Analysis

In [ ]:
# Time series analysis
print("📅 TIME SERIES ANALYSIS")
print("=" * 50)

# Create time-based features
train_full['Year'] = train_full['Date'].dt.year
train_full['Month'] = train_full['Date'].dt.month
train_full['Day'] = train_full['Date'].dt.day
train_full['WeekOfYear'] = train_full['Date'].dt.isocalendar().week
train_full['Quarter'] = train_full['Date'].dt.quarter

# Aggregate daily sales across all stores
daily_sales = train_full[train_full['Open'] == 1].groupby('Date')['Sales'].agg(['sum', 'mean', 'count'])
daily_sales.columns = ['Total_Sales', 'Avg_Sales', 'Open_Stores']

print("📊 Daily Sales Summary:")
print(daily_sales.describe().round(2))

In [ ]:
# Time series visualization
fig, axes = plt.subplots(3, 1, figsize=(15, 12))
fig.suptitle('Time Series Analysis', fontsize=16)

# Daily total sales
axes[0].plot(daily_sales.index, daily_sales['Total_Sales'], alpha=0.7, color='blue')
axes[0].set_title('Daily Total Sales Across All Stores')
axes[0].set_ylabel('Total Sales')
axes[0].grid(True, alpha=0.3)

# Weekly seasonality
weekly_sales = train_full[train_full['Open'] == 1].groupby('DayOfWeek')['Sales'].mean()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1].bar(range(7), weekly_sales, color='orange', alpha=0.7)
axes[1].set_title('Average Sales by Day of Week')
axes[1].set_ylabel('Average Sales')
axes[1].set_xticks(range(7))
axes[1].set_xticklabels(days)
axes[1].grid(True, alpha=0.3)

# Monthly trends
monthly_sales = train_full[train_full['Open'] == 1].groupby('Month')['Sales'].mean()
axes[2].plot(monthly_sales.index, monthly_sales.values, marker='o', linewidth=2, color='green')
axes[2].set_title('Average Sales by Month')
axes[2].set_ylabel('Average Sales')
axes[2].set_xlabel('Month')
axes[2].set_xticks(range(1, 13))
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print insights
print("\n🔍 TIME SERIES INSIGHTS:")
print(f"📅 Best day of week: {days[weekly_sales.idxmax() - 1]} (Sales: {weekly_sales.max():,.0f})")
print(f"📅 Worst day of week: {days[weekly_sales.idxmin() - 1]} (Sales: {weekly_sales.min():,.0f})")
print(f"📅 Best month: {monthly_sales.idxmax()} (Sales: {monthly_sales.max():,.0f})")
print(f"📅 Worst month: {monthly_sales.idxmin()} (Sales: {monthly_sales.min():,.0f})")

## 6. Store Analysis

In [ ]:
# Store performance analysis
print("🏪 STORE ANALYSIS")
print("=" * 50)

# Store-level aggregation
store_stats = train_full[train_full['Open'] == 1].groupby('Store').agg({
    'Sales': ['mean', 'std', 'sum', 'count'],
    'Customers': ['mean', 'sum'],
    'Promo': ['mean']
}).round(2)

store_stats.columns = ['Avg_Sales', 'Sales_Std', 'Total_Sales', 'Open_Days', 
                      'Avg_Customers', 'Total_Customers', 'Promo_Rate']

# Add store information
store_analysis = store_stats.merge(store_df, on='Store', how='left')

print("📊 Store Performance Summary:")
print(store_analysis.describe().round(2))

# Top and bottom performing stores
print("\n🏆 TOP 10 STORES (by average sales):")
top_stores = store_analysis.nlargest(10, 'Avg_Sales')[['Avg_Sales', 'StoreType', 'Assortment']]
display(top_stores)

print("\n⚠️ BOTTOM 10 STORES (by average sales):")
bottom_stores = store_analysis.nsmallest(10, 'Avg_Sales')[['Avg_Sales', 'StoreType', 'Assortment']]
display(bottom_stores)

## 7. Feature Engineering

In [ ]:
# Feature engineering for machine learning models
print("⚙️ FEATURE ENGINEERING")
print("=" * 50)

def create_features(df):
    """Create comprehensive feature set for modeling"""
    
    df = df.copy()
    
    # Time-based features
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week
    df['Quarter'] = df['Date'].dt.quarter
    df['DayOfYear'] = df['Date'].dt.dayofyear
    
    # Cyclical encoding for seasonal patterns
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['DayOfWeek_sin'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)
    df['DayOfWeek_cos'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7)
    
    # Competition features
    df['CompetitionDistance'].fillna(df['CompetitionDistance'].median(), inplace=True)
    df['CompetitionDistance_log'] = np.log1p(df['CompetitionDistance'])
    
    # Competition open duration
    df['CompetitionOpenSinceMonth'].fillna(0, inplace=True)
    df['CompetitionOpenSinceYear'].fillna(0, inplace=True)
    
    df['CompetitionOpen'] = ((df['CompetitionOpenSinceYear'] > 0) & 
                           (df['CompetitionOpenSinceMonth'] > 0)).astype(int)
    
    # Promo2 features
    df['Promo2SinceWeek'].fillna(0, inplace=True)
    df['Promo2SinceYear'].fillna(0, inplace=True)
    df['PromoInterval'].fillna('None', inplace=True)
    
    # Encode categorical variables
    df['StoreType_encoded'] = df['StoreType'].astype('category').cat.codes
    df['Assortment_encoded'] = df['Assortment'].astype('category').cat.codes
    
    # Holiday interactions
    df['Holiday_Promo'] = df['SchoolHoliday'] * df['Promo']
    df['StateHoliday_binary'] = (df['StateHoliday'] != '0').astype(int)
    
    return df

# Apply feature engineering
train_features = create_features(train_full)

print(f"✅ Original features: {train_full.shape[1]}")
print(f"✅ Enhanced features: {train_features.shape[1]}")
print(f"✅ New features added: {train_features.shape[1] - train_full.shape[1]}")

# List new features
new_features = [col for col in train_features.columns if col not in train_full.columns]
print(f"\n🆕 New features: {new_features}")

## 8. Model Training and Evaluation

In [ ]:
# Prepare data for modeling
print("🤖 MODEL TRAINING PREPARATION")
print("=" * 50)

# Filter open stores and non-zero sales for training
model_data = train_features[(train_features['Open'] == 1) & (train_features['Sales'] > 0)].copy()

print(f"📊 Model training data: {model_data.shape}")
print(f"📊 Sales range: {model_data['Sales'].min():,.0f} - {model_data['Sales'].max():,.0f}")

# Select features for modeling
feature_columns = [
    'Store', 'DayOfWeek', 'Promo', 'SchoolHoliday',
    'Year', 'Month', 'Day', 'WeekOfYear', 'Quarter', 'DayOfYear',
    'Month_sin', 'Month_cos', 'DayOfWeek_sin', 'DayOfWeek_cos',
    'StoreType_encoded', 'Assortment_encoded',
    'CompetitionDistance_log', 'CompetitionOpen',
    'Promo2', 'Holiday_Promo', 'StateHoliday_binary'
]

# Ensure all features exist
available_features = [col for col in feature_columns if col in model_data.columns]
print(f"\n📋 Available features: {len(available_features)}")
print(f"📋 Missing features: {set(feature_columns) - set(available_features)}")

X = model_data[available_features]
y = model_data['Sales']

print(f"\n✅ Feature matrix: {X.shape}")
print(f"✅ Target vector: {y.shape}")

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"📊 Training set: {X_train.shape}")
print(f"📊 Test set: {X_test.shape}")

# Define evaluation metrics
def evaluate_model(y_true, y_pred, model_name):
    """Calculate and display model performance metrics"""
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    results = {
        'Model': model_name,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE': mape
    }
    
    return results

# Store model results
model_results = []

In [ ]:
# 1. Linear Regression (Baseline)
print("🔵 Training Linear Regression (Baseline)...")

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

lr_pred = lr_model.predict(X_test)
lr_results = evaluate_model(y_test, lr_pred, 'Linear Regression')
model_results.append(lr_results)

print("✅ Linear Regression completed")
print(f"   RMSE: {lr_results['RMSE']:.2f}")
print(f"   R²: {lr_results['R²']:.4f}")

In [ ]:
# 2. Random Forest
print("🌳 Training Random Forest...")

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_results = evaluate_model(y_test, rf_pred, 'Random Forest')
model_results.append(rf_results)

print("✅ Random Forest completed")
print(f"   RMSE: {rf_results['RMSE']:.2f}")
print(f"   R²: {rf_results['R²']:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': available_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n🔍 Top 10 Most Important Features:")
display(feature_importance.head(10))

In [ ]:
# 3. XGBoost
print("🚀 Training XGBoost...")

xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_results = evaluate_model(y_test, xgb_pred, 'XGBoost')
model_results.append(xgb_results)

print("✅ XGBoost completed")
print(f"   RMSE: {xgb_results['RMSE']:.2f}")
print(f"   R²: {xgb_results['R²']:.4f}")

## 9. Model Comparison and Selection

In [ ]:
# Compare all models
print("📊 MODEL COMPARISON")
print("=" * 60)

results_df = pd.DataFrame(model_results)
results_df = results_df.round(4)
display(results_df)

# Find best model
best_model_idx = results_df['R²'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Model']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   R² Score: {results_df.loc[best_model_idx, 'R²']:.4f}")
print(f"   RMSE: {results_df.loc[best_model_idx, 'RMSE']:.2f}")
print(f"   MAPE: {results_df.loc[best_model_idx, 'MAPE']:.2f}%")

# Select best model for deployment
if best_model_name == 'Linear Regression':
    best_model = lr_model
    best_predictions = lr_pred
elif best_model_name == 'Random Forest':
    best_model = rf_model
    best_predictions = rf_pred
else:
    best_model = xgb_model
    best_predictions = xgb_pred

## 10. Model Validation and Insights

In [ ]:
# Model validation visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle(f'Model Validation - {best_model_name}', fontsize=16)

# Actual vs Predicted
axes[0, 0].scatter(y_test, best_predictions, alpha=0.5)
axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Actual Sales')
axes[0, 0].set_ylabel('Predicted Sales')
axes[0, 0].set_title('Actual vs Predicted')
axes[0, 0].grid(True, alpha=0.3)

# Residuals
residuals = y_test - best_predictions
axes[0, 1].scatter(best_predictions, residuals, alpha=0.5)
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Predicted Sales')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title('Residual Plot')
axes[0, 1].grid(True, alpha=0.3)

# Residuals histogram
axes[1, 0].hist(residuals, bins=50, alpha=0.7, color='orange')
axes[1, 0].set_xlabel('Residuals')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Residuals Distribution')
axes[1, 0].grid(True, alpha=0.3)

# Model comparison
models = results_df['Model']
r2_scores = results_df['R²']
axes[1, 1].bar(models, r2_scores, color=['lightblue', 'lightgreen', 'lightcoral'])
axes[1, 1].set_ylabel('R² Score')
axes[1, 1].set_title('Model Performance Comparison')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print validation insights
print("\n🔍 MODEL VALIDATION INSIGHTS:")
print(f"📊 Mean residual: {residuals.mean():.2f}")
print(f"📊 Std residual: {residuals.std():.2f}")
print(f"📊 Max absolute error: {np.abs(residuals).max():,.0f}")
print(f"📊 Median absolute error: {np.abs(residuals).median():,.0f}")

## 11. Save Model and Artifacts

In [ ]:
# Save model and preprocessing artifacts
print("💾 SAVING MODEL ARTIFACTS")
print("=" * 50)

import os
model_dir = Path('../src/models')
model_dir.mkdir(exist_ok=True)

# Save the best model
model_path = model_dir / 'best_model.joblib'
joblib.dump(best_model, model_path)
print(f"✅ Model saved: {model_path}")

# Save feature list
feature_list_path = model_dir / 'feature_list.joblib'
joblib.dump(available_features, feature_list_path)
print(f"✅ Feature list saved: {feature_list_path}")

# Save store information for predictions
store_info_path = model_dir / 'store_info.joblib'
joblib.dump(store_df, store_info_path)
print(f"✅ Store info saved: {store_info_path}")

# Save model metadata
model_metadata = {
    'model_name': best_model_name,
    'model_type': type(best_model).__name__,
    'features': available_features,
    'performance': results_df.loc[best_model_idx].to_dict(),
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'training_samples': len(X_train),
    'test_samples': len(X_test)
}

metadata_path = model_dir / 'model_metadata.joblib'
joblib.dump(model_metadata, metadata_path)
print(f"✅ Model metadata saved: {metadata_path}")

print("\n🎉 All model artifacts saved successfully!")
print(f"📁 Model directory: {model_dir.absolute()}")

## 12. Summary and Next Steps

In [ ]:
# Project summary
print("📋 PROJECT SUMMARY")
print("=" * 60)
print(f"🎯 Project: Rossmann Sales Forecasting")
print(f"📊 Dataset: {train_full.shape[0]:,} records, {len(store_df)} stores")
print(f"📅 Time period: {train_full['Date'].min()} to {train_full['Date'].max()}")
print(f"🏆 Best model: {best_model_name}")
print(f"📈 Model performance: R² = {results_df.loc[best_model_idx, 'R²']:.4f}")
print(f"🎯 RMSE: {results_df.loc[best_model_idx, 'RMSE']:.2f}")
print(f"🎯 MAPE: {results_df.loc[best_model_idx, 'MAPE']:.2f}%")

print("\n✅ COMPLETED TASKS:")
print("   ✓ Data exploration and quality assessment")
print("   ✓ Comprehensive EDA with time series analysis")
print("   ✓ Feature engineering with time-based features")
print("   ✓ Multiple model training and comparison")
print("   ✓ Model validation and performance evaluation")
print("   ✓ Model artifacts saved for deployment")

print("\n🚀 NEXT STEPS:")
print("   1. Export training logic to train.py script")
print("   2. Create Flask web service (predict.py)")
print("   3. Build Docker container")
print("   4. Deploy to cloud platform")
print("   5. Create comprehensive documentation")

print("\n🎉 Notebook completed successfully!")
print("Ready for production deployment phase.")